# Numpy and How Computers Work

Numpy is short for **Numerical Python** and is a package based around the **numpy array**.

The array is like the `list` container, except they're much more efficient on memory and operation speed. NumPy arrays form the core of nearly the entire ecosystem of data science tools in Python, so time spent learning to use NumPy well pays off in the long run.

In [1]:
import numpy as np

np.__version__

'1.18.5'

# Static and dynamic types

In the previous lecture, we saw how **file extensions** tell the computer how to interpret the data in a file.

Data types are the same thing. Since all data on a computer is 1's and 0's (binary numbers) anyway, we can get large speedups if we can assume the binary numbers are uniformly organised a certain way.

This contrasts with python's **dynamic typing**, where variables can be of any type

In [2]:
x = 5
x = "cat" # perfectly valid type change

**Statically-typed** languages like C or Java requires each variable to be explicitly declared, a dynamically-typed language like Python skips this specification. For example, in C you might specify a particular operation as follows:

```C
// C code
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```

In python this would be:

In [3]:
result = 0
for i in range(100):
    result += i

## A Python Integer Is More Than Just an Integer

The standard Python implementation is written in C.
This means that every Python object is simply a cleverly-disguised C structure, which contains not only its value, but other information as well. For example, when we define an integer in Python, such as ``x = 10000``, ``x`` is not just a "raw" integer. It's actually a pointer to a compound C structure, which contains several values.
Looking through the Python 3.4 source code, we find that the integer (long) type definition effectively looks like this (once the C macros are expanded):

```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```

A single integer in Python 3.4 actually contains four pieces:

- ``ob_refcnt``, a reference count that helps Python silently handle memory allocation and deallocation
- ``ob_type``, which encodes the type of the variable
- ``ob_size``, which specifies the size of the following data members
- ``ob_digit``, which contains the actual integer value that we expect the Python variable to represent.

This means that there is some overhead in storing an integer in Python as compared to an integer in a compiled language like C, as illustrated in the following figure:

Here ``PyObject_HEAD`` is the part of the structure containing the reference count, type code, and other pieces mentioned before.

Notice the difference here: a C integer is essentially a label for a position in memory whose bytes encode an integer value.
A Python integer is a pointer to a position in memory containing all the Python object information, including the bytes that contain the integer value.
This extra information in the Python integer structure is what allows Python to be coded so freely and dynamically.
All this additional information in Python types comes at a cost, however, which becomes especially apparent in structures that combine many of these objects.

## A Python List Is More Than Just a List

Let's consider now what happens when we use a Python data structure that holds many Python objects.
The standard mutable multi-element container in Python is the list.
We can create a list of integers as follows:

In [4]:
l = [str(c) for c in range(10)]
l[0 : 3] = [int(x) for x in l[0 : 3]]
l

[0, 1, 2, '3', '4', '5', '6', '7', '8', '9']

But this flexibility comes at a cost: to allow these flexible types, each item in the list must contain its own type info, reference count, and other information–that is, each item is a complete Python object.
In the special case that all variables are of the same type, much of this information is redundant: it can be much more efficient to store data in a fixed-type array.
The difference between a dynamic-type list and a fixed-type (NumPy-style) array is illustrated in the following figure:

![Array Memory Layout](array_vs_list.png)

At the implementation level, the array essentially contains a single pointer to one contiguous block of data.
The Python list, on the other hand, contains a pointer to a block of pointers, each of which in turn points to a full Python object like the Python integer we saw earlier.
Again, the advantage of the list is flexibility: because each list element is a full structure containing both data and type information, the list can be filled with data of any desired type.
Fixed-type NumPy-style arrays lack this flexibility, but are much more efficient for storing and manipulating data.

# Numpy Intro

NumPy arrays are somewhat like native Python lists, except that

- Data *must be homogeneous* (all elements of the same type).  
- These types must be one of the [data types](https://docs.scipy.org/doc/numpy/reference/arrays.dtypes.html) (`dtypes`) provided by NumPy:

| Data type	    | Description |
|---------------|-------------|
| ``bool_``     | Boolean (True or False) stored as a byte |
| ``int_``      | Default integer type (same as C ``long``; normally either ``int64`` or ``int32``)| 
| ``intc``      | Identical to C ``int`` (normally ``int32`` or ``int64``)| 
| ``intp``      | Integer used for indexing (same as C ``ssize_t``; normally either ``int32`` or ``int64``)| 
| ``int8``      | Byte (-128 to 127)| 
| ``int16``     | Integer (-32768 to 32767)|
| ``int32``     | Integer (-2147483648 to 2147483647)|
| ``int64``     | Integer (-9223372036854775808 to 9223372036854775807)| 
| ``uint8``     | Unsigned integer (0 to 255)| 
| ``uint16``    | Unsigned integer (0 to 65535)| 
| ``uint32``    | Unsigned integer (0 to 4294967295)| 
| ``uint64``    | Unsigned integer (0 to 18446744073709551615)| 
| ``float_``    | Shorthand for ``float64``.| 
| ``float16``   | Half precision float: sign bit, 5 bits exponent, 10 bits mantissa| 
| ``float32``   | Single precision float: sign bit, 8 bits exponent, 23 bits mantissa| 
| ``float64``   | Double precision float: sign bit, 11 bits exponent, 52 bits mantissa| 
| ``complex_``  | Shorthand for ``complex128``.| 
| ``complex64`` | Complex number, represented by two 32-bit floats| 
| ``complex128``| Complex number, represented by two 64-bit floats| 


There are also dtypes to represent complex numbers, unsigned integers, etc.

On modern machines, the default dtype for arrays is `float64`

First, we can use ``np.array`` to create arrays from Python lists:

In [5]:
import numpy as np

# integer array:
a = np.array([1, 4, 2, 5, 3])
print(a.dtype)
a

int64


array([1, 4, 2, 5, 3])

### Upcasting

Remember that unlike Python lists, NumPy is constrained to arrays that all contain the same type.
If types do not match, NumPy will upcast if possible (here, integers are up-cast to floating point):

In [6]:
a = np.array([3.14, 4, 2, 3])
print(a.dtype)
a

float64


array([3.14, 4.  , 2.  , 3.  ])

If we want to explicitly set the data type of the resulting array, we can use the ``dtype`` keyword:

In [7]:
np.array([1, 2, 3, 4], dtype='float32')

array([1., 2., 3., 4.], dtype=float32)

The hierarchy looks like

`bool < int < float < str < object`

with sub hierarchies within types like `int8 < int16 < int32 < int64`

### PyObject fallback

If numpy fails to find a common type for elements, it will fallback to the `object` type at which point it's basically a list

In [8]:
np.array([1, 'cat', 3.5]) # <U21 is a string type (unicode)

array(['1', 'cat', '3.5'], dtype='<U21')

# Array Shape

Unlike Python lists, NumPy arrays can explicitly be multi-dimensional; here's one way of initializing a multidimensional array using a list of lists:

In [9]:
# nested lists are multi-dimensional arrays
np.array([[2,3,4], [6,7,8], [9, 10, 11]])

array([[ 2,  3,  4],
       [ 6,  7,  8],
       [ 9, 10, 11]])

However, if our matrix is not rectangular (each row having the same length), numpy will fallback to holding each as an `object`, which is no better than python's lists.

In [10]:
# Falls back to object
np.array([[2,3,4,5,6,7,8,9,1,2,3], 
          [6,7,8], 
          [9, 10, 11]]
)

array([list([2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3]), list([6, 7, 8]),
       list([9, 10, 11])], dtype=object)

# Array Creation



In [11]:
# Create a length-10 integer array filled with zeros
np.zeros(10, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [12]:
# Create a 3x5 floating-point array filled with ones
np.ones((3, 5), dtype=float)

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [43]:
# Create a 3x5 array filled with the number pi
np.full((3, 5), np.pi, dtype=float)

array([[3.14159265, 3.14159265, 3.14159265, 3.14159265, 3.14159265],
       [3.14159265, 3.14159265, 3.14159265, 3.14159265, 3.14159265],
       [3.14159265, 3.14159265, 3.14159265, 3.14159265, 3.14159265]])

numpy also has `np.arange` which is its analogue to python's `range`:

In [14]:
# Create an array filled with a linear sequence
# Starting at 3, ending at 20, stepping by 2
# (this is similar to the built-in range() function)
np.arange(3, 20, 2)

array([ 3,  5,  7,  9, 11, 13, 15, 17, 19])

### np.empty

The function `np.empty` reserves a block of memory on your computer to be an array, but doesn't touch it.

Since the memory at that space could have been anything before we reserved it, the values in it can be any kind of garbage:

In [46]:
np.empty(10, dtype=np.int64)

array([     93848146501872,                   0,   76562361913315848,
       8144812737470923018,  148619896090538504,   72628274983469328,
       3028672991414190356,  -68115792473346552,  360908130047623938,
                  34603280])

### Random arrays

In [60]:
# Create a 3x3 array of uniformly distributed
print(np.random)
# random values between 0 and 1
np.random.random((2,2))

<module 'numpy.random' from '/home/sadnan/anaconda3/lib/python3.8/site-packages/numpy/random/__init__.py'>


array([[0.7793203 , 0.13601551],
       [0.76903227, 0.10172184]])

In [17]:
# Create a 3x3 array of random integers in the interval [0, 10)
np.random.randint(0, 10, (3, 3))

array([[6, 1, 4],
       [0, 4, 4],
       [3, 8, 0]])

# Advanced Indexing

In [58]:
x = np.array([
    [3, 5, 2, 4],
    [7, 6, 8, 8],
    [1, 6, 7, 9]]
)
# We index (row, column)
x[0, 0]

3

In [59]:
x[2, -1]

9

In [20]:
x[0, 0] = 9999
x

array([[9999,    5,    2,    4],
       [   7,    6,    8,    8],
       [   1,    6,    7,    7]])


Keep in mind that, unlike Python lists, NumPy arrays have a fixed type.
This means, for example, that if you attempt to insert a floating-point value to an integer array, the value will be silently truncated. Don't be caught unaware by this behavior!

In [21]:
x[0, 0] = np.pi  # silently truncated!
x

array([[3, 5, 2, 4],
       [7, 6, 8, 8],
       [1, 6, 7, 7]])

Indexing works as `[start : stop : stride]`

In [67]:
x = np.arange(50)
print(x)
x[::3]  # every third element
print(x[::3])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
[ 0  3  6  9 12 15 18 21 24 27 30 33 36 39 42 45 48]


In [68]:
x[-25 : -5 : 2] # not including the item at index -5

array([25, 27, 29, 31, 33, 35, 37, 39, 41, 43])

Negative steps give us an easy way to reverse an array:

In [70]:
print(x)
reverse_x = x[::-1]
print(reverse_x)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
[49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26
 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10  9  8  7  6  5  4  3  2
  1  0]


The comma separates row indexing from column indexing

`[row start : stop : step, col start : stop : step]`

In [73]:
x = np.random.randint(0, 10, (5, 10))
x

array([[2, 1, 6, 2, 4, 5, 3, 6, 3, 2],
       [1, 4, 7, 5, 6, 0, 7, 7, 4, 9],
       [8, 7, 5, 8, 9, 6, 6, 8, 3, 3],
       [3, 7, 0, 0, 6, 8, 9, 9, 0, 2],
       [6, 8, 3, 3, 3, 1, 6, 3, 2, 9]])

In [78]:
x[:2, :3] # two rows (0 and 1), three columns (0, 1 ,2)

array([[2, 1, 6],
       [1, 4, 7]])

In [27]:
x[:, ::2]  # all rows, every other column

array([[0, 8, 1, 4, 9],
       [8, 5, 0, 9, 4],
       [8, 6, 7, 7, 4],
       [9, 3, 8, 5, 0],
       [0, 2, 0, 8, 9]])

In [81]:
x[:, 3] # just the 4th column

array([2, 5, 8, 0, 3])

### Subarrays as no-copy views

One important–and extremely useful–thing to know about array slices is that they return *views* rather than *copies* of the array data.
This is one area in which NumPy array slicing differs from Python list slicing: in lists, slices will be copies.

Let's extract a $2 \times 2$ subarray from this:

In [84]:
x_sub = x[:2, :2]
x_sub

array([[999999,      1],
       [     1,      4]])

In [86]:
x_sub[0, 0] = 999999 # Changes x
x

array([[999999,      1,      6,      2,      4,      5,      3,      6,
             3,      2],
       [     1,      4,      7,      5,      6,      0,      7,      7,
             4,      9],
       [     8,      7,      5,      8,      9,      6,      6,      8,
             3,      3],
       [     3,      7,      0,      0,      6,      8,      9,      9,
             0,      2],
       [     6,      8,      3,      3,      3,      1,      6,      3,
             2,      9]])

### Copying arrays

Is done with np.copy() or the `a.copy()` syntax

In [37]:
x2 = x.copy()
x2[:] = -1
x

array([1, 2, 3])

### Concatenation of arrays

Concatenation, or joining of two arrays in NumPy, is primarily accomplished using the routines ``np.concatenate``, ``np.vstack``, and ``np.hstack``.
``np.concatenate`` takes a tuple or list of arrays as its first argument, as we can see here:

In [38]:
x = np.array([1, 2, 3])
y = np.array([3, 2, 1])
np.concatenate([x, y])

array([1, 2, 3, 3, 2, 1])

You can also concatenate more than two arrays at once:

In [39]:
z = [99, 99, 99]
print(np.concatenate([x, y, z]))

[ 1  2  3  3  2  1 99 99 99]


It can also be used for two-dimensional arrays:

In [40]:
grid = np.array([[1, 2, 3],
                 [4, 5, 6]])
# concatenate along the first axis
np.concatenate([grid, grid])

array([[1, 2, 3],
       [4, 5, 6],
       [1, 2, 3],
       [4, 5, 6]])

For working with arrays of mixed dimensions, it can be clearer to use the ``np.vstack`` (vertical stack) and ``np.hstack`` (horizontal stack) functions:

In [92]:
x = np.array([1, 2, 3])
grid = np.array([[9, 8, 7],
                 [6, 5, 4]])

# vertically stack the arrays
res = np.vstack([x, grid])
# print(res)
# print(x) 
# print(grid)

# horizontally stack the arrays
y = np.array([[99],
              [99]])
res = np.hstack([grid, y])
print(res)

[[ 9  8  7 99]
 [ 6  5  4 99]]


### Note: appending is slow

Adding elements to an existing array can be don in numpy, but it requires re-allocating chunks of memory on the computer, so is very slow.

If you care about performance, you should always build the whole array first, then fill it